# Tensorflow mutiple category classifier model with IRIS dataset

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
import tensorflow as tf

In [2]:
data = sns.load_dataset("iris")

In [3]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [5]:
from sklearn.model_selection import train_test_split

In [1]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [5]:
## its an other approach to read the data where tensorflow converts the categorical values to numerical automatically


train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0s/step


In [11]:
X_train=train
X_test=test
Y_train=X_train.pop("Species")
Y_test= X_test.pop("Species")

In [12]:
for i in train.keys():
    print(i)

SepalLength
SepalWidth
PetalLength
PetalWidth


In [10]:
for i in train.columns:
    print(i)

SepalLength
SepalWidth
PetalLength
PetalWidth
Species


In [17]:
### create feature columns as 

feature_columns=[]

for col in X_train.columns:
    feature_columns.append(tf.feature_column.numeric_column(col,dtype=tf.float64))
    

In [18]:
feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float64, normalizer_fn=None)]

In [37]:
### create input function, use lambda concept

def input_func(features, labels, training=True,batch_size=256):
    res = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    
     # Shuffle and repeat if you are in training mode.
    if training:
        res = res.shuffle(1000).repeat()
    
    return res.batch(batch_size)

In [31]:
### now create an estimator

model = tf.estimator.DNNClassifier(
    ### feature columns 
    feature_columns=feature_columns,
    
    ## it represents the no of output classes
    n_classes=3,
    
    hidden_units =[10,20,30]
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\vamsee\\AppData\\Local\\Temp\\tmpywpck99t', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [32]:
model.train(
input_fn= lambda: input_func(X_train,Y_train), steps=5000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\vamsee\AppData\Local\Temp\tmpywpck99t\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.1215439, step = 0
INFO:tensorflow:global_step/sec: 249.937
INFO:tensorflow:loss = 1.0761478, step = 100 (0.417 sec)
INFO:tensorflow:global_step/sec: 315.421
INFO:tensorflow:loss = 1.051738, step = 200 (0.319 sec)
INFO:tensorflow:global_step/sec: 300.681
INFO:tensorflow:loss = 1.0272124, step = 300 (0.326 sec)
INFO:tensorflow:global_step/sec: 308.204
INFO:tensorflow:loss = 0.99163634, step = 400 (0.330 sec)
INFO:tensorflow:global_step/sec: 305.458
INFO:tensorflow:loss = 0.9623547,

In [38]:
### now evaluate the model as follows

res= model.evaluate(input_fn=lambda: input_func(X_test,Y_test,False ))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-02-04T11:33:50
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\vamsee\AppData\Local\Temp\tmpywpck99t\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.27816s
INFO:tensorflow:Finished evaluation at 2022-02-04-11:33:51
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.7, average_loss = 0.65659344, global_step = 5000, loss = 0.65659344
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\vamsee\AppData\Local\Temp\tmpywpck99t\model.ckpt-5000


In [79]:
### lets build a predict model


def input_fc(inputs):
    res = tf.data.Dataset.from_tensor_slices(dict(inputs)).batch(256)
    return res

inputs={}
for features in X_train.columns:
    
    flag=True
    
    while flag:
        val=input("enter {}".format(features))
        
        if not val.isdigit(): flag=False
    inputs[features]= [float(val)]
    
res = model.predict(input_fn=lambda: input_fc(predict_x))
 



enter SepalLength 1.1
enter SepalWidth 2.2
enter PetalLength 3.3
enter PetalWidth 4.4


In [82]:
for i in res:
    print(i["class_ids"][0])

In [78]:
for pred_dict in res:
    class_id = pred_dict['class_ids'][0]
    print(pred_dict)
    print(class_id)
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\vamsee\AppData\Local\Temp\tmpywpck99t\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([ 2.3316545 , -0.63042676, -2.3094394 ], dtype=float32), 'probabilities': array([0.94218886, 0.04872169, 0.00908942], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2]), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
0
Prediction is "Setosa" (94.2%)
{'logits': array([-0.08633064,  0.2496356 ,  0.13372627], dtype=float32), 'probabilities': array([0.27431536, 0.38384724, 0.34183747], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2]), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
1
Prediction is "Versicolor" 

In [72]:
res

<generator object Estimator.predict at 0x00000295FBBB79E0>